In [1]:
import numpy as np
import pandas as pd
import os
import gc
import xgboost as xgb
from xgboost import plot_importance
import math

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import time
import warnings

warnings.filterwarnings('ignore')

In [2]:
def get_psi(train,test,f_cols):
    psi_res = pd.DataFrame()
    psi_dict={}
    for c in tqdm(f_cols):
        try:
            t_train = train[c].fillna(-998)
            t_test = test[c].fillna(-998)
            #获取切分点
            bins=[]
            for i in np.arange(0,1.1,0.2):
                bins.append(t_train.quantile(i))
            bins=sorted(set(bins))
            bins[0]=-np.inf
            bins[-1]=np.inf
            #计算psi
            t_psi = pd.DataFrame()
            t_psi['train'] = pd.cut(t_train,bins).value_counts().sort_index()
            t_psi['test'] = pd.cut(t_test,bins).value_counts()
            if c == 'outdoorTemp':
                print(t_psi['train'])
                print(t_psi['test'])
            t_psi.index=[str(x) for x in t_psi.index]
            t_psi.loc['总计',:] = t_psi.sum()
            t_psi['train_rate'] = t_psi['train']/t_psi.loc['总计','train']
            t_psi['test_rate'] = t_psi['test']/t_psi.loc['总计','test']
            t_psi['psi'] = (t_psi['test_rate']-t_psi['train_rate'])*(np.log(t_psi['test_rate'])-np.log(t_psi['train_rate']))
            t_psi.loc['总计','psi'] = t_psi['psi'].sum()
            t_psi.index.name=c
            #汇总
            t_res = pd.DataFrame([[c,t_psi.loc['总计','psi']]],
                                 columns=['变量名','PSI'])
            psi_res = pd.concat([psi_res,t_res])
            psi_dict[c]=t_psi
            print(c,'done')
        except:
            print(c,'error')
    return psi_res,psi_dict

In [3]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
sub = pd.DataFrame(test_df['time'])

In [4]:
train_df = train_df[train_df['temperature'].notnull()]
train_df = train_df.fillna(method='bfill')
test_df = test_df.fillna(method='bfill')

In [5]:
train_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                    'indoorHum', 'indoorAtmo', 'temperature']
test_df.columns = ['time', 'year', 'month', 'day', 'hour', 'min', 'sec', 'outdoorTemp', 'outdoorHum', 'outdoorAtmo',
                   'indoorHum', 'indoorAtmo']

In [6]:
print('train_df.shape: ', train_df.shape)
train_df = train_df.loc[(train_df['outdoorTemp'] >= test_df['outdoorTemp'].min()) & (train_df['outdoorTemp'] <= test_df['outdoorTemp'].max())]
print('处理后 train_df.shape: ', train_df.shape)

train_df.shape:  (24807, 13)
处理后 train_df.shape:  (19338, 13)


In [7]:
data_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [8]:
# data_df['indoorAtmo-outdoorAtmo'] = data_df['indoorAtmo'] - data_df['outdoorAtmo']
# data_df['indoorHum-outdoorHum'] = data_df['indoorHum'] - data_df['outdoorHum']

In [9]:
# 基本聚合特征
group_feats = []
for f in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']):
    data_df['MDH_{}_medi'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('median')
    data_df['MDH_{}_mean'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('mean')
    data_df['MDH_{}_max'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('max')
    data_df['MDH_{}_min'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('min')
    data_df['MDH_{}_sum'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('sum')
    data_df['MDH_{}_std'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('std')
    data_df['MDH_{}_skew'.format(f)] = data_df.groupby(['month', 'day', 'hour'])[f].transform('skew')
    
    data_df['MD_{}_medi'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('median')
    data_df['MD_{}_mean'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('mean')
    data_df['MD_{}_max'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('max')
    data_df['MD_{}_min'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('min')
    data_df['MD_{}_sum'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('sum')
    data_df['MD_{}_std'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('std')
    data_df['MD_{}_skew'.format(f)] = data_df.groupby(['month', 'day'])[f].transform('skew')

    group_feats.append('MDH_{}_medi'.format(f))
    group_feats.append('MDH_{}_mean'.format(f))
    group_feats.append('MDH_{}_max'.format(f))
    group_feats.append('MDH_{}_min'.format(f))
    group_feats.append('MDH_{}_sum'.format(f))
    
    group_feats.append('MD_{}_medi'.format(f))
    group_feats.append('MD_{}_mean'.format(f))
    group_feats.append('MD_{}_max'.format(f))
    group_feats.append('MD_{}_min'.format(f))
    group_feats.append('MD_{}_sum'.format(f))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  3.43it/s]


In [10]:
# 基本交叉特征
for f1 in tqdm(['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo'] + group_feats):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo'] + group_feats:
        if f1 != f2:
            colname_substract = '{}_{}_subtract'.format(f1, f2)
            colname_add = '{}_{}_add'.format(f1, f2)            
            colname_multiply = '{}_{}_multyply'.format(f1, f2)
            colname_ratio = '{}_{}_ratio'.format(f1, f2)
            data_df[colname_substract] = data_df[f1].values - data_df[f2].values
            data_df[colname_add] = data_df[f1].values + data_df[f2].values
            data_df[colname_multiply] = data_df[f1].values / data_df[f2].values
            data_df[colname_ratio] = data_df[f1].values * data_df[f2].values            

100%|██████████████████████████████████████████████████████████████████████████████████| 55/55 [02:07<00:00,  2.32s/it]


In [11]:
data_df = data_df.fillna(method='bfill')

In [12]:
# 历史信息提取
data_df['dt'] = data_df['day'].values + (data_df['month'].values - 3) * 31

for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo', 'temperature']:
    tmp_df = pd.DataFrame()
    for t in tqdm(range(15, 45)):
        tmp = data_df.loc[data_df['dt'] < t, :].groupby(['hour'], as_index=False)[f].agg({
            'hit_{}_mean'.format(f): 'mean',
            'hit_{}_median'.format(f): 'median',
            'hit_{}_max'.format(f): 'max',
            'hit_{}_min'.format(f): 'min',
            'hit_{}_sum'.format(f): 'sum',
            'hit_{}_std'.format(f): 'std',
            'hit_{}_skew'.format(f): 'skew'
        })
        tmp['dt'] = t
        tmp_df = tmp_df.append(tmp)
        del tmp
        gc.collect()

    data_df = data_df.merge(tmp_df, on=['dt', 'hour'], how='left')
    del tmp_df
    gc.collect()

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:28<00:00,  1.06it/s]


In [13]:
data_df = data_df.fillna(method='bfill')

In [14]:
# 离散化
for f in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
    data_df[f + '_20_bin'] = pd.cut(data_df[f], 20, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_50_bin'] = pd.cut(data_df[f], 50, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_100_bin'] = pd.cut(data_df[f], 100, duplicates='drop').apply(lambda x: x.left).astype(int)
    data_df[f + '_200_bin'] = pd.cut(data_df[f], 200, duplicates='drop').apply(lambda x: x.left).astype(int)

In [15]:
for f1 in tqdm(['outdoorTemp_20_bin', 'outdoorHum_20_bin', 'outdoorAtmo_20_bin', 'indoorHum_20_bin', 'indoorAtmo_20_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')

for f1 in tqdm(['outdoorTemp_50_bin', 'outdoorHum_50_bin', 'outdoorAtmo_50_bin', 'indoorHum_50_bin', 'indoorAtmo_50_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')

for f1 in tqdm(['outdoorTemp_100_bin', 'outdoorHum_100_bin', 'outdoorAtmo_100_bin', 'indoorHum_100_bin',
                'indoorAtmo_100_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')

for f1 in tqdm(['outdoorTemp_200_bin', 'outdoorHum_200_bin', 'outdoorAtmo_200_bin', 'indoorHum_200_bin',
                'indoorAtmo_200_bin']):
    for f2 in ['outdoorTemp', 'outdoorHum', 'outdoorAtmo', 'indoorHum', 'indoorAtmo']:
        data_df['{}_{}_medi'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('median')
        data_df['{}_{}_mean'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('mean')
        data_df['{}_{}_max'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('max')
        data_df['{}_{}_min'.format(f1, f2)] = data_df.groupby([f1])[f2].transform('min')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:41<00:00, 44.33s/it]


In [16]:
train_count = train_df.shape[0]
train_df = data_df[:train_count].copy().reset_index(drop=True)
test_df = data_df[train_count:].copy().reset_index(drop=True)
del data_df
gc.collect()

29

In [17]:
drop_columns = ["time", "year", "sec", "temperature"]

features = train_df[:1].drop(drop_columns, axis=1).columns
x_train = train_df[features]
x_test = test_df[features]

In [18]:
y_train = train_df['temperature'].values - train_df['outdoorTemp'].values

In [19]:
psi_res, psi_dict = get_psi(x_train, x_test, features)

  0%|                                                                                | 8/12422 [00:00<05:20, 38.68it/s]

month done
day done
hour done
min done
(-inf, 14.8]    4103
(14.8, 16.2]    3733
(16.2, 17.8]    4050
(17.8, 20.4]    3678
(20.4, inf]     3774
Name: train, dtype: int64
(-inf, 14.8]     47
(14.8, 16.2]     42
(16.2, 17.8]     47
(17.8, 20.4]     57
(20.4, inf]     213
Name: test, dtype: int64
outdoorTemp done
outdoorHum done
outdoorAtmo done
indoorHum done


  0%|                                                                               | 16/12422 [00:00<05:24, 38.20it/s]

indoorAtmo done
MDH_outdoorTemp_medi done
MDH_outdoorTemp_mean done
MDH_outdoorTemp_max done
MDH_outdoorTemp_min done
MDH_outdoorTemp_sum done
MDH_outdoorTemp_std done
MDH_outdoorTemp_skew done


  0%|▏                                                                              | 24/12422 [00:00<05:18, 38.97it/s]

...

indoorHum_200_bin_indoorAtmo_medi done
indoorHum_200_bin_indoorAtmo_mean done
indoorHum_200_bin_indoorAtmo_max done
indoorHum_200_bin_indoorAtmo_min done
indoorAtmo_200_bin_outdoorTemp_medi done
indoorAtmo_200_bin_outdoorTemp_mean done
indoorAtmo_200_bin_outdoorTemp_max done
indoorAtmo_200_bin_outdoorTemp_min done
indoorAtmo_200_bin_outdoorHum_medi done


100%|███████████████████████████████████████████████████████████████████████████▉| 12413/12422 [05:31<00:00, 40.06it/s]

indoorAtmo_200_bin_outdoorHum_mean done
indoorAtmo_200_bin_outdoorHum_max done
indoorAtmo_200_bin_outdoorHum_min done
indoorAtmo_200_bin_outdoorAtmo_medi done
indoorAtmo_200_bin_outdoorAtmo_mean done
indoorAtmo_200_bin_outdoorAtmo_max done
indoorAtmo_200_bin_outdoorAtmo_min done
indoorAtmo_200_bin_indoorHum_medi done
indoorAtmo_200_bin_indoorHum_mean done


100%|████████████████████████████████████████████████████████████████████████████| 12422/12422 [05:31<00:00, 37.49it/s]

indoorAtmo_200_bin_indoorHum_max done
indoorAtmo_200_bin_indoorHum_min done
indoorAtmo_200_bin_indoorAtmo_medi done
indoorAtmo_200_bin_indoorAtmo_mean done
indoorAtmo_200_bin_indoorAtmo_max done
indoorAtmo_200_bin_indoorAtmo_min done


In [20]:
features = list(psi_res[psi_res['PSI'] <= 0.2]['变量名'].values) + ['outdoorTemp']

In [21]:
x_train = x_train[features]
x_test = x_test[features]

In [22]:
nums = int(x_train.shape[0] * 0.8)

trn_x, trn_y, val_x, val_y = x_train[:nums], y_train[:nums], x_train[nums:], y_train[nums:]

train_matrix = xgb.DMatrix(trn_x, label=trn_y, missing=np.nan)
valid_matrix = xgb.DMatrix(val_x, label=val_y, missing=np.nan)
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

params = {
    'booster': 'gbtree',
    'eval_metric': 'mae',
    'min_child_weight': 5,
    'max_depth': 8,
    'subsample': 0.5,
    'colsample_bytree': 0.5,
    'eta': 0.01,
    'seed': 2020,
    'nthread': 36,
    'silent': 1
}

watchlist = [(train_matrix, 'train'), (valid_matrix, 'eval')]

model_eval = xgb.train(params,
                       train_matrix,
                       num_boost_round=50000,
                       evals=watchlist,
                       verbose_eval=500,
                       early_stopping_rounds=1000)
val_pred = model_eval.predict(valid_matrix, ntree_limit=model_eval.best_ntree_limit).reshape(-1, 1)

[0]	train-mae:0.72617	eval-mae:0.31623
Multiple eval metrics have been passed: 'eval-mae' will be used for early stopping.

Will train until eval-mae hasn't improved in 1000 rounds.
[500]	train-mae:0.05197	eval-mae:0.16175
[1000]	train-mae:0.04258	eval-mae:0.16143
[1500]	train-mae:0.03632	eval-mae:0.16159
Stopping. Best iteration:
[609]	train-mae:0.04861	eval-mae:0.16126



In [23]:
mse = mean_squared_error(val_y, val_pred)
print("mse_score:", mse)
print("mse_score:", str(mse)[2:7])

mse_score: 0.04319695195399701
mse_score: 04319


In [24]:
feat_imp_dict = model_eval.get_score(importance_type='gain')

In [25]:
feat_imp = pd.Series(feat_imp_dict).sort_values(ascending=False).reset_index().rename(columns={'index': 'feature',
                                                                                               0: 'importance'})
feat_imp['normalized_importance'] = feat_imp['importance'] / feat_imp['importance'].sum()
feat_imp['cumulative_importance'] = np.cumsum(feat_imp['normalized_importance'])
record_low_importance = feat_imp[feat_imp['cumulative_importance'] > 0.95]

to_drop = list(record_low_importance['feature'])
print(to_drop)

['MDH_outdoorTemp_mean_MD_indoorAtmo_medi_add', 'MDH_outdoorHum_min_indoorHum_subtract', 'MDH_indoorHum_max_outdoorAtmo_ratio', 'outdoorHum_MDH_indoorAtmo_mean_multyply', 'MDH_indoorHum_min_MD_outdoorAtmo_medi_add', 'MDH_indoorAtmo_max_MD_indoorAtmo_medi_multyply', 'MDH_indoorAtmo_mean_outdoorHum_subtract', 'MDH_outdoorAtmo_mean_MD_outdoorHum_medi_subtract', 'MDH_indoorHum_medi_MD_indoorHum_medi_ratio', 'MDH_outdoorAtmo_min_MDH_indoorHum_mean_multyply', 'MDH_indoorHum_mean_MD_outdoorHum_max_subtract', 'outdoorAtmo_MDH_outdoorHum_mean_multyply', 'MDH_indoorHum_medi_MD_outdoorAtmo_medi_add', 'MDH_outdoorTemp_max_MD_indoorAtmo_medi_add', 'MD_outdoorHum_medi_MDH_indoorAtmo_medi_subtract', 'MD_outdoorTemp_mean_MDH_indoorHum_max_add', 'MDH_outdoorHum_medi_outdoorHum_subtract', 'indoorHum_MDH_outdoorHum_min_ratio', 'MDH_outdoorHum_medi_outdoorTemp_subtract', 'MD_outdoorAtmo_medi_indoorHum_subtract', 'MD_indoorHum_mean_MDH_indoorAtmo_medi_subtract', 'indoorHum_MDH_outdoorAtmo_mean_multyply', '

In [26]:
feat_imp.head()

,feature,importance,normalized_importance,cumulative_importance
0,MDH_indoorHum_mean_MDH_outdoorHum_mean_subtract,82.794076,0.059781,0.059781
1,MDH_outdoorHum_mean_MDH_indoorHum_mean_subtract,70.735864,0.051074,0.110855
2,MDH_outdoorHum_mean_MDH_indoorHum_mean_multyply,57.136907,0.041255,0.152110
3,indoorHum_outdoorHum_multyply,46.468917,0.033552,0.185662
4,MDH_indoorHum_medi_MDH_outdoorHum_mean_subtract,41.488899,0.029957,0.215619


In [27]:
x_train.drop(to_drop, axis=1, inplace=True)
x_test.drop(to_drop, axis=1, inplace=True)

In [28]:
train_all_matrix = xgb.DMatrix(x_train, y_train, missing=np.nan)
test_matrix = xgb.DMatrix(x_test, label=val_y, missing=np.nan)

model = xgb.train(params,
                  train_all_matrix,
                  num_boost_round=model_eval.best_ntree_limit + 20)

test_pred = model.predict(test_matrix, ntree_limit=model.best_ntree_limit).reshape(-1, 1)

In [29]:
sub['temperature'] = test_pred[:, 0] + test_df['outdoorTemp'].values
sub.to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), str(mse)[2:7]), index=False)